# %% [markdown]
# # Análisis de Utilización de Transcripciones
# 
# Este notebook carga un fichero CSV con líneas en el formato:
# 
# ```
# ep498_B_whisper_es.html,Héctor Socas,00:41:09.56
# ep498_B_whisper_es.html,Francis Villatoro,00:48:40.78
# ep498_B_whisper_es.html,Gastón Giribet,00:29:51.14
# ep498_B_whisper_es.html,Isabel Cordero,00:01:43.82
# ```
# 
# y genera gráficas para visualizar la utilización de cada hablante.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from datetime import datetime, timedelta


In [ ]:
# 1. Cargar el CSV
SPFILE = '/home/jmrobles/Podcasts/Coffee Break/speakingtimes.csv'
df = pd.read_csv(SPFILE, header=None, names=['archivo', 'hablante', 'tiempo'])
df.head()

In [ ]:
# 2. Convertir la columna 'tiempo' a objetos timedelta
def convertir_a_timedelta(tiempo_str):
    try:
        tiempo_obj = datetime.strptime(tiempo_str, '%H:%M:%S.%f').time()
        return timedelta(hours=tiempo_obj.hour, minutes=tiempo_obj.minute, seconds=tiempo_obj.second, microseconds=tiempo_obj.microsecond)
    except ValueError:
        try:
            tiempo_obj = datetime.strptime(tiempo_str, '%M:%S.%f').time()
            return timedelta(minutes=tiempo_obj.minute, seconds=tiempo_obj.second, microseconds=tiempo_obj.microsecond)
        except ValueError:
            try:
                tiempo_obj = datetime.strptime(tiempo_str, '%S.%f').time()
                return timedelta(seconds=tiempo_obj.second, microseconds=tiempo_obj.microsecond)
            except ValueError:
                return timedelta(0)

df['tiempo_timedelta'] = df['tiempo'].apply(convertir_a_timedelta)
df = df[1:]
df.head()

In [ ]:
# Incluir hablantes Unknown y ?? en la misma categoría
df['hablante'] = df['hablante'].apply(lambda x: 'Sin asignar' if str(x).startswith('???') or str(x).startswith('Unknown') else x)
df.head()

In [ ]:
# Agrupar por hablante y sumar los tiempos
totales_por_hablante = df.groupby('hablante')['tiempo_timedelta'].sum().reset_index()
totales_por_hablante['tiempo_segundos'] = totales_por_hablante['tiempo_timedelta'].dt.total_seconds()

# Cálculo de tiempo total y tiempo sin asignar
tiempo_total_acumulado_segundos = totales_por_hablante['tiempo_segundos'].sum()
sin_asignar_segundos = totales_por_hablante.loc[totales_por_hablante['hablante'] == 'Sin asignar', 'tiempo_segundos'].sum()

print(f'Tiempo total acumulado: {tiempo_total_acumulado_segundos} segundos')
print(f'Tiempo sin asignar: {sin_asignar_segundos} segundos')


In [ ]:


# Eliminar la categoría 'Sin asignar' para el gráfico
totales_por_hablante = totales_por_hablante[totales_por_hablante['hablante'] != 'Sin asignar']

# Ordenar por tiempo total
totales_por_hablante = totales_por_hablante.sort_values('tiempo_segundos', ascending=False)

# Crear el gráfico de barras horizontales
plt.figure(figsize=(12, 10))
bars = plt.barh(totales_por_hablante['hablante'], totales_por_hablante['tiempo_segundos'], color='skyblue')

# Formatear el eje x para mostrar el tiempo en formato legible
def segundos_a_hhmmss(segundos):
    horas = int(segundos // 3600)
    minutos = int((segundos % 3600) // 60)
    segundos = int(segundos % 60)
    return f'{horas:02d}:{minutos:02d}:{segundos:02d}'

plt.gca().xaxis.set_major_formatter(plt.FuncFormatter(lambda x, _: segundos_a_hhmmss(x)))

# Añadir etiquetas y título
plt.xlabel('Tiempo total')
plt.ylabel('Hablan')
plt.title('Tiempo total por hablante')

# Mostrar los valores de tiempo a la derecha de las barras
for bar, segundos in zip(bars, totales_por_hablante['tiempo_segundos']):
    plt.text(bar.get_width() + 10, bar.get_y() + bar.get_height() / 2, segundos_a_hhmmss(segundos), va='center')

# Calcular el tiempo total acumulado y el tiempo sin asignar en formato hh:mm:ss
tiempo_total_acumulado_hhmmss = segundos_a_hhmmss(tiempo_total_acumulado_segundos)
sin_asignar_hhmmss = segundos_a_hhmmss(sin_asignar_segundos)

# Añadir anotación con el tiempo total acumulado y el tiempo sin asignar
plt.text(0.95, 0.95, f'Tiempo total: {tiempo_total_acumulado_hhmmss}\nSin asignar: {sin_asignar_hhmmss} ({sin_asignar_segundos/tiempo_total_acumulado_segundos*100:.2f}%)', color='black', fontsize=12,
         transform=plt.gca().transAxes, ha='right', va='top')

# 13. Mostrar el gráfico y guardarlo en un archivo
# plt.show()
plt.savefig('speakingtime.png')